In [1]:
%%bash

pip install --upgrade pip
pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss-gpu]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-nhnzhagp/farm-haystack_4c3ebdd946874613aaa78d3eacdee928
  Resolved https://github.com/deepset-ai/haystack.git to commit 75ef9596780a18b4c5ec83777aeede1aef53be89
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━

DEPRECATION: git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss-gpu] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-nhnzhagp/farm-haystack_4c3ebdd946874613aaa78d3eacdee928


In [2]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.WARNING)


In [3]:
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(embedding_dim=768, faiss_index_factory_str="Flat")

In [5]:
import os
print(os.getcwd())
os.chdir('./drive/MyDrive/pale-companion-files/')

/content


In [6]:
# Import some files
import pickle
with open('./chapter_fmt_list.pkl','rb') as f:
    chapters = pickle.load(f)

# Temporary
# chapters = chapters[:100]

In [7]:
from haystack import Document
chapter_documents = [Document.from_dict(d) for d in chapters]
len(chapter_documents)

307

In [8]:
from haystack.nodes import PreProcessor

word_preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
    split_overlap=20,
    progress_bar=True, 
    add_page_number=True
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
docs_word = word_preprocessor.process(chapter_documents)
len(docs_word)

Preprocessing:   0%|          | 0/307 [00:00<?, ?docs/s]

53227

In [10]:
# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs_word)

Writing Documents:   0%|          | 0/53227 [00:00<?, ?it/s]

In [12]:
from haystack.nodes import DensePassageRetriever

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    embed_title=False
    # could sub the models for the facebook ones with larget embedding dims for testing
)

document_store.update_embeddings(retriever)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


Updating Embedding:   0%|          | 0/53227 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/3232 [00:00<?, ? Docs/s]

In [13]:
document_store.save("pale_lfqa_v1_fb.faiss")

In [14]:
from haystack.utils import print_documents
from haystack.pipelines import DocumentSearchPipeline

p_retrieval = DocumentSearchPipeline(retriever)
res = p_retrieval.run(query="Tell me something about Avery?", params={"Retriever": {"top_k": 10}})
print_documents(res, max_text_len=512)



Query: Tell me something about Avery?

{   'content': 'Avery took the harder route, around the head of the table, '
               'hugging her grandfather and getting the money from her dad '
               'before finding her way around.  Sheridan backed her chair up '
               'an inch or two to get in Avery’s way.  “You’re more childish '
               'than some of the actual children at the table,” Lucy said.  '
               '“And you’re a jerk.”  “Not so much of a jerk right now as I am '
               'a bad influence,” Lucy said.  “What?” Sheridan asked.  Lucy '
               'discreetly pointed toward the pitcher of water with ice '
               'cubes.  She saw Avery’s look of r...',
    'name': None}

{   'content': 'And mom and dad, and even bite size Declan, all giving me one '
               'hundred percent of their pity, their attention, cooing over '
               'me, and all I wanted was to be a grumpy preteen, and they '
               'wouldn’t let 

In [35]:
from haystack.nodes import Seq2SeqGenerator


generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")


In [36]:
from haystack.pipelines import GenerativeQAPipeline

pipe = GenerativeQAPipeline(generator, retriever)


In [37]:
pipe.run(query="Tell me something about Avery?", params={"Retriever": {"top_k": 10}})


{'query': 'Tell me something about Avery?',
 'answers': [<Answer {'answer': "I don't know much about Avery, but I can tell you that she has a lot of energy. She likes to be active, she likes to do things, and she loves to have fun.", 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['efda1b975d883ca730681722ac885797', '5a526e65d774f66d5970a5043028a8fb', '482ba9330bf9e70ab4fe7e38d9900b1a', '12bb7a27b618f43c9ac4cc0663009096', '3f5ddc11336af2b6c043c4b382ce5ef1', 'b0c79c04589ceac29ca85d8aaccedb6c', '7c11b2a7bb0f08fc5b592d7af37e09cd', 'fe55e618a3de0891566d285b888bfdea', 'd3e952a768a0dc382b345996be52dde4', '1f196cb4ed9a5cf2e89498cca8fd7b34'], 'meta': {'doc_scores': [0.6787846477047164, 0.6785674080510622, 0.6774731500714143, 0.6774143338179758, 0.6773957108343794, 0.677214586719268, 0.6771212191532069, 0.6770147248590306, 0.6769374950005875, 0.6767252091717976], 'content': ['Avery took the harder route, around the 